# Obligatorio
<h3>Técnicas de Machine Learning para análisis de datos</h3>

Máster en Big Data - Universidad ORT Uruguay

**Estudiante:**
- Sebastián Uriarte Güimil, N° de estudiante: 194973

**Nota:** La versión más reciente de este obligatorio, así como de los
laboratorios anteriores, se encuentran disponible en [este repositorio](https://github.com/sebassu/tecnicas-machine-learning).

### 1. Introducción

El presente obligatorio, entregable final del curso, tiene como objetivo la
aplicación de los distintos conceptos, técnicas y modelos vistos como parte del
contenido de la materia, al análisis de un caso real.

En particular, el  _dataset_ propuesto corresponde a la Encuesta Continua de
Hogares en Uruguay 2022, ya visto en tareas anteriores. Se buscará aplicar
técnicas de aprendizaje automático sobre este para en esta ocasión intentar
predecir en función de sus datos socioeconómicos si una persona es pobre o no,
tratándose por tanto un problema de clasificación.

Asimismo, se excluirán del proceso de entrenamiento datos correspondientes a
ingresos o egresos (gastos) de la persona, buscándose en cambio estudiar la
relación entre las restantes variables y el nivel de pobreza, de forma de
complementar su definición mediante el método del ingreso; es decir, por la
insuficiencia de recursos para acceder a un grupo de necesidades básicas.[¹](https://www.gub.uy/instituto-nacional-estadistica/sites/instituto-nacional-estadistica/files/documentos/publicaciones/Informe%20Linea%20de%20Pobreza%202006%20Final.pdf)

Los resultados del trabajo llevado a cabo se exponen a continuación.



### 2. Análisis exploratorio

**Carga de los datos**

In [1]:
import pandas as pd

In [71]:
data = pd.read_csv(r'data/ECH_2022.csv')
data.head()

/var/folders/x7/n6t8px653mj0l89x730r03540000gp/T/ipykernel_23648/3348143444.py:1: DtypeWarning: Columns (84,179,278) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'data/ECH_2022.csv')


,ID,nper,anio,mes,GR,region,region_4,dpto,ccz,secc,...,w_tri,w_sem,estred13,nom_dpto,g260,g261,g261_1,w_ano,Loc_agr_13,Nom_loc_agr_13
0,11558,1,2022,1,7,1,1,1,1,4,...,175,87.5,4,MONTEVIDEO,NaN,NaN,NaN,38,1010,Montevideo
1,11559,1,2022,1,7,1,1,1,1,4,...,350,175.0,4,MONTEVIDEO,NaN,NaN,NaN,84,1010,Montevideo
2,11559,2,2022,1,7,1,1,1,1,4,...,350,175.0,4,MONTEVIDEO,NaN,NaN,NaN,84,1010,Montevideo
3,11560,1,2022,1,7,1,1,1,1,4,...,131,65.5,4,MONTEVIDEO,NaN,NaN,NaN,33,1010,Montevideo
4,11561,1,2022,1,7,1,1,1,1,4,...,131,65.5,4,MONTEVIDEO,NaN,NaN,NaN,33,1010,Montevideo


In [72]:
data.shape

(55056, 528)

Vemos entonces que el _dataset_ original es uno bastante grande, con 528 columnas,
y más de cincuenta mil filas. De esas columnas, seleccionaremos las que resultarán
útiles para el problema a abordar, excluyéndose las que la propuesta menciona como
no utilizables, y aquellas que tengan que ver con IDs, etc.

In [73]:
columns_to_select = [
    "dpto", "secc", "segm", "ccz", "region", "region_4", "estred13", "c1", "c2",
    "c3", "c4", "c6", "d8_1", "d9", "d10", "d11", "d12", "d13", "d14", "d15",
    "d16", "d18", "d260", "d19", "d20", "d21_1", "d21_2", "d21_3", "d21_4",
    "d21_4_1", "d21_5", "d21_5_1", "d21_6", "d21_20", "d21_7", "d21_10",
    "d21_11", "d21_12", "d21_13", "d21_14", "d21_14_1", "d21_15", "d21_15_1",
    "d21_15_2", "d21_15_3", "d21_15_4", "d21_15_5", "d21_15_6", "d21_16",
    "d21_16_1", "d21_16_2", "d21_21", "d21_17", "d21_18", "d21_18_1", "d21_19",
    "d21_19_1", "d181", "d229", "d230", "d231", "d232", "d184", "d184_1", "d23", 
    "d24", "d25", "h155", "h156", "h252", "h158_1", "h158_2", "h159", "h160",
    "h161", "h162", "h227", "h269", "h167_1", "h167_2", "h167_3", "h167_4",
    "h169", "h271", "h171", "h171_2", "h172", "h173", "i228", "i259", "HT19",
    "pobre"
]

data = data[columns_to_select]
data.head()

,dpto,secc,segm,ccz,region,region_4,estred13,c1,c2,c3,...,h169,h271,h171,h171_2,h172,h173,i228,i259,HT19,pobre
0,1,4,103,1,1,1,4,3,1,1,...,2,2,2,0,2,2,2,2,1,0
1,1,4,103,1,1,1,4,3,1,1,...,2,2,2,0,2,2,2,2,2,0
2,1,4,103,1,1,1,4,3,1,1,...,2,2,2,0,2,2,2,2,2,0
3,1,4,103,1,1,1,4,3,1,1,...,2,2,2,0,2,2,2,2,1,0
4,1,4,103,1,1,1,4,3,1,1,...,2,2,2,0,2,2,2,2,1,0


Esta información, así como la explicación de a que contenido corresponde cada
código de columna, se obtuvo del archivo `DICCIONARIO ECH 2022_propuesto.xlsx`,
presentado como parte de la propuesta del obligatorio.

In [74]:
print(data.isnull().any().any())
set(data.dtypes)

False


{dtype('int64')}

Con esto verificamos que, dentro de las columnas seleccionadas, no existen
celdas nulas, lo cual hace innecesario el tener que rellenar valores faltantes.
Por otro lado, correspondería verificar y ajustar de ser necesario el tipo de
cada una de estas columnas (que, como podemos ver arriba, se han cargado todas
con el tipo `int64`), de la siguiente forma:

In [75]:
for column in ["dpto", "secc", "segm", "ccz", "estred13", "d8_1", "d11"]:
    data[column] = data[column].astype('category')

for column in ["d15", "d21_1", "d21_2", "d21_3", "d21_4", "d21_5", "d21_6",
               "d21_20", "d21_7", "d21_10", "d21_11", "d21_12", "d21_13",
               "d21_14", "d21_15", "d21_15_1", "d21_15_3", "d21_15_5", "d21_16",
               "d21_16_1", "d21_16_2", "d21_21", "d21_17", "d21_18", "d21_19",
               "d181", "d231", "d184", "h155", "h156", "h252", "h159", "h160",
               "h161", "h162", "h227", "h269", "h167_2", "h167_3", "h167_4",
               "h169", "h271", "h171", "h167_2", "h167_3", "h167_4", "h169",
               "h271", "h171", "h172", "h173", "i228", "i259"]:
    data[column] = data[column].replace({1: True, 2: False}).astype(bool)

three_dtype = pd.CategoricalDtype(categories=list(range(1, 4)), ordered=True)
data['region'] = data['region'].astype(three_dtype)
data['d13'] = data['d13'].astype(three_dtype)
data['d19'] = data['d19'].astype(three_dtype)
data['h167_1'] = data['h167_1'].astype(three_dtype)

four_dtype = pd.CategoricalDtype(categories=list(range(1, 5)), ordered=True)
data['region_4'] = data['region_4'].astype(four_dtype)
data['d12'] = data['d12'].astype(four_dtype)
data['d16'] = data['d16'].astype(four_dtype)
data['d18'] = data['d18'].astype(four_dtype)

five_dtype = pd.CategoricalDtype(categories=list(range(1, 6)), ordered=True)
data['c1'] = data['c1'].astype(five_dtype)
data['c4'] = data['c4'].astype(five_dtype)

six_dtype = pd.CategoricalDtype(categories=list(range(1, 7)), ordered=True)
data['c2'] = data['c2'].astype(six_dtype)
data['c3'] = data['c3'].astype(six_dtype)
data['d260'] = data['d260'].astype(six_dtype)
data['d20'] = data['d20'].astype(six_dtype)

data['pobre'] = data['pobre'].astype(bool)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55056 entries, 0 to 55055
Data columns (total 92 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dpto      55056 non-null  category
 1   secc      55056 non-null  category
 2   segm      55056 non-null  category
 3   ccz       55056 non-null  category
 4   region    55056 non-null  category
 5   region_4  55056 non-null  category
 6   estred13  55056 non-null  category
 7   c1        55056 non-null  category
 8   c2        55056 non-null  category
 9   c3        55056 non-null  category
 10  c4        55056 non-null  category
 11  c6        55056 non-null  int64   
 12  d8_1      55056 non-null  category
 13  d9        55056 non-null  int64   
 14  d10       55056 non-null  int64   
 15  d11       55056 non-null  category
 16  d12       55056 non-null  category
 17  d13       55056 non-null  category
 18  d14       55056 non-null  int64   
 19  d15       55056 non-null  bool    
 20  d16   

In [76]:
data.describe()

,c6,d9,d10,d14,d21_4_1,d21_5_1,d21_14_1,d21_15_2,d21_15_4,d21_15_6,...,d230,d232,d184_1,d23,d24,d25,h158_1,h158_2,h171_2,HT19
count,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,...,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000,55056.000000
mean,1.006884,3.585077,2.182142,1.214454,0.143581,1.518799,0.770670,0.421516,0.732400,0.050658,...,1.334496,0.001726,0.631757,2.485469,0.669137,3.154606,0.645670,0.600843,0.024920,3.157821
std,0.082684,1.188153,0.821709,0.543168,0.413908,0.967143,1.066753,0.767812,1.006224,0.250681,...,6.768846,0.043637,4.192585,1.120498,0.986633,1.564489,0.627782,0.946715,0.235363,1.564394
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,3.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000
50%,1.000000,3.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,0.000000,0.000000,3.000000
75%,1.000000,4.000000,3.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,3.000000,1.000000,4.000000,1.000000,2.000000,0.000000,4.000000
max,2.000000,14.000000,7.000000,6.000000,4.000000,8.000000,10.000000,6.000000,8.000000,5.000000,...,168.000000,2.000000,99.000000,10.000000,8.000000,12.000000,7.000000,12.000000,7.000000,12.000000


Como se ha mencionado anteriormente, el objetivo de este obligatorio es entrenar
modelos que, tomando como entrada los datos socioeconómicos de una persona,
permitan predecir si esta es pobre o no. Esto se encuentra codificado en el
_dataset_ bajo la columna `pobre`, de tipo booleano (`False` si no es pobre, y
`True` si sí lo es). Este es entonces el problema de clasificación a abordar.

En cuanto a eso, resultará interesante determinar cuantos ejemplos de una u otra
clase se encuentran en el dataset. Tenemos entonces:
